In [4]:
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
import pickle
import re

In [5]:
c = pd.DataFrame()
c = pd.read_feather("/Users/huilingchen/Documents/GitHub/indeed_web_scraper/source/11_23_2020")
c.size()

TypeError: 'numpy.int64' object is not callable

In [6]:
import os

In [9]:
list = os.listdir("/Users/huilingchen/Documents/GitHub/indeed_web_scraper/source") # dir is your directory path
number_files = len(list)
number_files

5

In [13]:
import glob
dir_name = "/Users/huilingchen/Documents/GitHub/indeed_web_scraper/source"
merged_df = pd.DataFrame()
for filename in os.listdir(dir_name):
    temp_df = pd.read_feather(filename)
    merged_df.append(temp_df)
merged_df.size()

ArrowInvalid: Not a Feather V1 or Arrow IPC file